Importamos las librerias necesarias para hacer el scrapping

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

### Web Scraping

Primero vamos a establecer la URL base de la cual vamos a tomar todos los datos. Tomando la primera por orden alfabético (Andalucía - Almería), tiene la forma:

https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/andalucia/almeria-provincia/historico/    

  es decir:  

 url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/" + comunidad + provincia + "historico/"

Creamos primero un diccionario para las comunidades y provincias. Nos servirá más tarde para organizar y agrupar los datos.  
También creamos uno que siga el formato que pide idealista

In [ ]:
diccionario_provincias =     {
    "Andalucia": ["Almeria", "Cadiz", "Cordoba", "Granada", "Huelva", "Jaen", "Malaga", "Sevilla"],
    "Aragon": ["Huesca", "Teruel", "Zaragoza"],
    "Principado de Asturias": ["Asturias"],
    "Islas Baleares": ["Islas Baleares"],
    "Canarias": ["Las Palmas", "Santa Cruz de Tenerife"],
    "Cantabria": ["Cantabria"],
    "Castilla y León": ["Avila", "Burgos", "Leon", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"],
    "Castilla-La Mancha": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"],
    "Cataluña": ["Barcelona", "Girona", "Lleida", "Tarragona"],
    "Extremadura": ["Badajoz", "Caceres"],
    "Galicia": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
    "Comunidad de Madrid": ["Madrid"],
    "Región de Murcia": ["Murcia"],
    "Navarra": ["Navarra"],
    "La Rioja": ["La Rioja"],
    "País Vasco": ["Alava", "Guipuzcoa", "Vizcaya"],
    "Valencia": ["Alicante", "Castellon", "Valencia"],
    "Ceuta": ["Ceuta"],
    "Melilla": ["Melilla"]
}

diccionario_provincias_idealista =     {
    "andalucia": ["almeria-provincia", "cadiz-provincia", "cordoba-provincia", "granada-provincia", "huelva-provincia", "jaen-provincia", "malaga-provincia", "sevilla-provincia"],
    "aragon": ["huesca-provincia", "teruel-provincia", "zaragoza-provincia"],
    "asturias": ["asturias"],
    "baleares": ["baleares"],
    "canarias": ["las-palmas", "santa-cruz-de-tenerife-provincia"],
    "cantabria": ["cantabria"],
    "castilla-y-leon": ["avila-provincia", "burgos-provincia", "leon-provincia", "palencia-provincia", "salamanca-provincia", "segovia-provincia", "soria-provincia", "valladolid-provincia", "zamora-provincia"],
    "castilla-la-mancha": ["albacete-provincia", "ciudad-real-provincia", "cuenca-provincia", "guadalajara-provincia", "toledo-provincia"],
    "cataluna": ["barcelona-provincia", "girona-provincia", "lleida-provincia", "tarragona-provincia"],
    "extremadura": ["badajoz-provincia", "caceres-provincia"],
    "galicia": ["a-coruna-provincia", "lugo-provincia", "ourense-provincia", "pontevedra-provincia"],
    "madrid-comunidad": ["madrid-provincia"],
    "murcia-region": ["murcia-provincia"],
    "navarra": ["navarra"],
    "la-rioja": ["la-rioja"],
    "euskadi": ["alava", "guipuzcoa", "vizcaya"],
    "comunitat-valenciana": ["alicante-alacant", "castellon-castello", "valencia-valencia"],
    "ceuta": ["ceuta-provincia"],
    "melilla": ["melilla-provincia"]
}

Incluyo los headers y cookies de una búsqueda manual hecha con Chrome. De no hacer esto, me detecta como un bot  
y la consulta devuelve un error o una página de captcha

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
    "Accept-Language": "es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Connection": "keep-alive",
}

cookies = {
    "cookie_name1": "_pprv=eyJjb25zZW50Ijp7IjAiOnsibW9kZSI6Im9wdC1pbiJ9LCIxIjp7Im1vZGUiOiJvcHQtaW4ifSwiMiI6eyJtb2RlIjoib3B0LWluIn0sIjMiOnsibW9kZSI6Im9wdC1pbiJ9LCI0Ijp7Im1vZGUiOiJvcHQtaW4ifSwiNSI6eyJtb2RlIjoib3B0LWluIn0sIjYiOnsibW9kZSI6Im9wdC1pbiJ9LCI3Ijp7Im1vZGUiOiJvcHQtaW4ifX0sInB1cnBvc2VzIjpudWxsLCJfdCI6Im1hZnJpcTFnfGx1cmNsOHBnIn0%3D; _cb=DtdneiCvzmhkD2aYPi; _chartbeat2=.1712604649306.1712604649306.1.F5BgGBhETAisW8YxpcH9KDyjBDJ.1; _pcid=%7B%22browserId%22%3A%22lurcl8pd26v7baqq%22%2C%22_t%22%3A%22mafriqr7%7Clurcl9f7%22%7D; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXSwH18yBbQgDMY9AI4wAHAB9UYGAGNUATiGSQAXyA; didomi_token=eyJ1c2VyX2lkIjoiMThlYmYzMTEtNDU2YS02NjhiLWFmNTItYTE5NzI3MWQzZTZlIiwiY3JlYXRlZCI6IjIwMjQtMDQtMDhUMTk6MzA6NDguNTM0WiIsInVwZGF0ZWQiOiIyMDI0LTA0LTA4VDE5OjMwOjUyLjg1MFoiLCJ2ZW5kb3JzIjp7ImRpc2FibGVkIjpbImdvb2dsZSIsImM6bGlua2VkaW4tbWFya2V0aW5nLXNvbHV0aW9ucyIsImM6bWl4cGFuZWwiLCJjOmFidGFzdHktTExrRUNDajgiLCJjOmhvdGphciIsImM6eWFuZGV4bWV0cmljcyIsImM6YmVhbWVyLUg3dHI3SGl4IiwiYzphcHBzZmx5ZXItR1VWUExwWVkiLCJjOnRlYWxpdW1jby1EVkRDZDhaUCIsImM6dGlrdG9rLUtaQVVRTFo5IiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmlkZWFsaXN0YS1MenRCZXFFMyIsImM6aWRlYWxpc3RhLWZlUkVqZTJjIiwiYzpjb250ZW50c3F1YXJlIiwiYzptaWNyb3NvZnQiXX0sInB1cnBvc2VzIjp7ImRpc2FibGVkIjpbImFuYWx5dGljcy1IcEJKcnJLNyIsImdlb2xvY2F0aW9uX2RhdGEiLCJkZXZpY2VfY2hhcmFjdGVyaXN0aWNzIl19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFBQUEuQUFBQSJ9; euconsent-v2=CP8wAMAP8wAMAAHABBENAvEgAAAAAAAAAAAAAAAAAACkoAMAAQUIKQAYAAgoQQgAwABBQgdABgACChASADAAEFCA.YAAAAAAAAAAA; utag_main__sn=2; utag_main_ses_id=1733165620668%3Bexp-session; utag_main__ss=0%3Bexp-session; utag_main__prevCompleteClickName=; utag_main__prevVtUrl=https%3A%2F%2Fwww.idealista.com%2Fsala-de-prensa%2Finformes-precio-vivienda%2Fventa%2Fmadrid-comunidadmadrid-provinciahistorico%2F%3Bexp-1733172915379; utag_main__prevVtUrlReferrer=%3Bexp-1733172915379; utag_main__prevVtSource=Direct traffic%3Bexp-1733172915379; utag_main__prevVtCampaignName=organicWeb%3Bexp-1733172915379; utag_main__prevVtCampaignCode=%3Bexp-1733172915379; utag_main__prevVtCampaignLinkName=%3Bexp-1733172915379; utag_main__prevVtRecipientId=%3Bexp-1733172915379; utag_main__prevVtProvider=%3Bexp-1733172915379; dicbo_id=%7B%22dicbo_fetch%22%3A1733171512240%7D; utag_main__pn=63%3Bexp-session; utag_main__se=66%3Bexp-session; utag_main__st=1733173315636%3Bexp-session; utag_main__prevCompletePageName=255-idealista/others > portal > https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/aragon/huesca-provincia/%3Bexp-1733175115651; utag_main__prevLevel2=255-idealista/others%3Bexp-1733175115651; datadome=mUdSCn2DSVG3xVodDcBWqGkxbQJxlduMfSuQhns3QqEtgpgrgVk1L5F7o2xpV22oeLCeAxTlmlL8afn8Wx6R91jb0JHP4QDMvfNhjZCqOg1coguNifrx8wY7SSxxIpWE",
    
    
}

In [4]:
df = pd.DataFrame(columns = ["Comunidad", "Provincia", "Fecha","Precio"])

for comunidad in diccionario_provincias_idealista.keys():
    for provincia in diccionario_provincias_idealista[comunidad]:
        
        url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/" + comunidad + "/" + provincia + "/historico/"
        #Ponemos que espere 5 segundos entre requests para que no crea que soy un bot
        time.sleep(5)

        response_es = requests.get(url=url, headers=headers, cookies=cookies)
        sopa = BeautifulSoup(response_es.text, "lxml")
              
        try:
            for tabla in sopa.find("tbody").children:
                nueva_fila = pd.DataFrame({
                    "Comunidad" : [comunidad],
                    "Provincia" : [provincia],
                    "Fecha" : [list(tabla)[0].text],
                    "Precio" : [str(list(tabla)[1].text)]

                })

                df = pd.concat([df, nueva_fila], ignore_index=True)
        except:
                 nueva_fila = pd.DataFrame({
                    "Comunidad" : [comunidad],
                    "Provincia" : [provincia],
                    "Fecha" : "-",
                    "Precio" : "-"})
                 df = pd.concat([df, nueva_fila], ignore_index=True)

df

,Comunidad,Provincia,Fecha,Precio
0,andalucia,almeria-provincia,Noviembre 2024,1.297 €/m2
1,andalucia,almeria-provincia,Octubre 2024,1.296 €/m2
2,andalucia,almeria-provincia,Septiembre 2024,1.279 €/m2
3,andalucia,almeria-provincia,Agosto 2024,1.273 €/m2
4,andalucia,almeria-provincia,Julio 2024,1.262 €/m2
...,...,...,...,...
11799,melilla,melilla-provincia,Mayo 2006,n.d.
11800,melilla,melilla-provincia,Abril 2006,n.d.
11801,melilla,melilla-provincia,Marzo 2006,n.d.
11802,melilla,melilla-provincia,Febrero 2006,n.d.


In [ ]:
df.to_csv("./data/datasets_brutos/csv_idealista_venta.csv", sep=";", header = True)

Repetimos ahora con el alquiler. Solo hay que cambiar la parte de la url correspondiente de "venta" a "alquiler"

In [6]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
    "Accept-Language": "es,es-ES;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Connection": "keep-alive",
}

cookies = {
    "cookie_name1": "_pprv=eyJjb25zZW50Ijp7IjAiOnsibW9kZSI6Im9wdC1pbiJ9LCIxIjp7Im1vZGUiOiJvcHQtaW4ifSwiMiI6eyJtb2RlIjoib3B0LWluIn0sIjMiOnsibW9kZSI6Im9wdC1pbiJ9LCI0Ijp7Im1vZGUiOiJvcHQtaW4ifSwiNSI6eyJtb2RlIjoib3B0LWluIn0sIjYiOnsibW9kZSI6Im9wdC1pbiJ9LCI3Ijp7Im1vZGUiOiJvcHQtaW4ifX0sInB1cnBvc2VzIjpudWxsLCJfdCI6Im1hZnJpcTFnfGx1cmNsOHBnIn0%3D; _cb=DtdneiCvzmhkD2aYPi; _chartbeat2=.1712604649306.1712604649306.1.F5BgGBhETAisW8YxpcH9KDyjBDJ.1; _pcid=%7B%22browserId%22%3A%22lurcl8pd26v7baqq%22%2C%22_t%22%3A%22mafriqr7%7Clurcl9f7%22%7D; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXSwH18yBbQgDMY9AI4wAHAB9UYGAGNUATiGSQAXyA; didomi_token=eyJ1c2VyX2lkIjoiMThlYmYzMTEtNDU2YS02NjhiLWFmNTItYTE5NzI3MWQzZTZlIiwiY3JlYXRlZCI6IjIwMjQtMDQtMDhUMTk6MzA6NDguNTM0WiIsInVwZGF0ZWQiOiIyMDI0LTA0LTA4VDE5OjMwOjUyLjg1MFoiLCJ2ZW5kb3JzIjp7ImRpc2FibGVkIjpbImdvb2dsZSIsImM6bGlua2VkaW4tbWFya2V0aW5nLXNvbHV0aW9ucyIsImM6bWl4cGFuZWwiLCJjOmFidGFzdHktTExrRUNDajgiLCJjOmhvdGphciIsImM6eWFuZGV4bWV0cmljcyIsImM6YmVhbWVyLUg3dHI3SGl4IiwiYzphcHBzZmx5ZXItR1VWUExwWVkiLCJjOnRlYWxpdW1jby1EVkRDZDhaUCIsImM6dGlrdG9rLUtaQVVRTFo5IiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmlkZWFsaXN0YS1MenRCZXFFMyIsImM6aWRlYWxpc3RhLWZlUkVqZTJjIiwiYzpjb250ZW50c3F1YXJlIiwiYzptaWNyb3NvZnQiXX0sInB1cnBvc2VzIjp7ImRpc2FibGVkIjpbImFuYWx5dGljcy1IcEJKcnJLNyIsImdlb2xvY2F0aW9uX2RhdGEiLCJkZXZpY2VfY2hhcmFjdGVyaXN0aWNzIl19LCJ2ZXJzaW9uIjoyLCJhYyI6IkFBQUEuQUFBQSJ9; euconsent-v2=CP8wAMAP8wAMAAHABBENAvEgAAAAAAAAAAAAAAAAAACkoAMAAQUIKQAYAAgoQQgAwABBQgdABgACChASADAAEFCA.YAAAAAAAAAAA; utag_main__sn=2; utag_main_ses_id=1733165620668%3Bexp-session; utag_main__ss=0%3Bexp-session; utag_main__prevCompleteClickName=; utag_main__prevVtUrl=https%3A%2F%2Fwww.idealista.com%2Fsala-de-prensa%2Finformes-precio-vivienda%2Fventa%2Fmadrid-comunidadmadrid-provinciahistorico%2F%3Bexp-1733172915379; utag_main__prevVtUrlReferrer=%3Bexp-1733172915379; utag_main__prevVtSource=Direct traffic%3Bexp-1733172915379; utag_main__prevVtCampaignName=organicWeb%3Bexp-1733172915379; utag_main__prevVtCampaignCode=%3Bexp-1733172915379; utag_main__prevVtCampaignLinkName=%3Bexp-1733172915379; utag_main__prevVtRecipientId=%3Bexp-1733172915379; utag_main__prevVtProvider=%3Bexp-1733172915379; dicbo_id=%7B%22dicbo_fetch%22%3A1733171512240%7D; utag_main__pn=63%3Bexp-session; utag_main__se=66%3Bexp-session; utag_main__st=1733173315636%3Bexp-session; utag_main__prevCompletePageName=255-idealista/others > portal > https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/aragon/huesca-provincia/%3Bexp-1733175115651; utag_main__prevLevel2=255-idealista/others%3Bexp-1733175115651; datadome=mUdSCn2DSVG3xVodDcBWqGkxbQJxlduMfSuQhns3QqEtgpgrgVk1L5F7o2xpV22oeLCeAxTlmlL8afn8Wx6R91jb0JHP4QDMvfNhjZCqOg1coguNifrx8wY7SSxxIpWE",
    
    
}

In [7]:
df = pd.DataFrame(columns = ["Comunidad", "Provincia", "Fecha","Precio"])

for comunidad in diccionario_provincias_idealista.keys():
    for provincia in diccionario_provincias_idealista[comunidad]:
        
        url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/" + comunidad + "/" + provincia + "/historico/"
        #Ponemos que espere 5 segundos entre requests para que no crea que soy un bot
        time.sleep(5)

        response_es = requests.get(url=url, headers=headers, cookies=cookies)
        sopa = BeautifulSoup(response_es.text, "lxml")
              
        try:
            for tabla in sopa.find("tbody").children:
                nueva_fila = pd.DataFrame({
                    "Comunidad" : [comunidad],
                    "Provincia" : [provincia],
                    "Fecha" : [list(tabla)[0].text],
                    "Precio" : [str(list(tabla)[1].text)]

                })

                df = pd.concat([df, nueva_fila], ignore_index=True)
        except:
                 nueva_fila = pd.DataFrame({
                    "Comunidad" : [comunidad],
                    "Provincia" : [provincia],
                    "Fecha" : "-",
                    "Precio" : "-"})
                 df = pd.concat([df, nueva_fila], ignore_index=True)

df

,Comunidad,Provincia,Fecha,Precio
0,andalucia,almeria-provincia,Noviembre 2024,"7,8 €/m2"
1,andalucia,almeria-provincia,Octubre 2024,"7,7 €/m2"
2,andalucia,almeria-provincia,Septiembre 2024,"7,7 €/m2"
3,andalucia,almeria-provincia,Agosto 2024,"7,9 €/m2"
4,andalucia,almeria-provincia,Julio 2024,"8,1 €/m2"
...,...,...,...,...
11799,melilla,melilla-provincia,Mayo 2006,n.d.
11800,melilla,melilla-provincia,Abril 2006,n.d.
11801,melilla,melilla-provincia,Marzo 2006,n.d.
11802,melilla,melilla-provincia,Febrero 2006,n.d.


In [ ]:
df.to_csv("./data/datasets_brutos/csv_idealista_alquiler.csv", sep=";", header = True)